In [ ]:

import pandas as pd
import numpy as np 
import re 
import nltk
nltk.download("punkt")
from nltk.corpus import stopwords 
sw=stopwords.words('english') 
from nltk.util import ngrams
from lib.config import connstr
from lib.utils import * 
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore') 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression 


In [ ]:
USE_CACHE=False
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [160]:
if not USE_CACHE:
    df=sql_todf(f"""
        SELECT FK_QuestionType, QuestionText
        FROM fsma_Questions WHERE FK_QuestionType IS NOT NULL AND QuestionText IS NOT NULL 
        AND FK_QuestionType IN (1,2 )
        """, connstr)   
    df['NQuestionText']=df['QuestionText'].apply(lambda s: re.sub('[^a-z\s]','',s.lower() )) 
    df['NQuestionText'].dropna(inplace=True)
    df['NQuestionText']=df['NQuestionText'].apply(lambda s: ' '.join([s for s in s.split() if s.lower() not in sw]).upper()) 

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['NQuestionText'])  
     
    dff = pd.DataFrame(columns=vectorizer.get_feature_names(), data=X.toarray()) 
    m=pd.merge(df, dff, left_index=True, right_index=True)
    m.to_csv(r'c:\temp\cache.csv', index=False)
df=pd.read_csv(r'c:\temp\cache.csv')

In [170]:
dff=df.copy()
print(dff.loc[820:855, ['FK_QuestionType', 'NQuestionText']])
 

     FK_QuestionType                                      NQuestionText
820                1   ENSURES RESOURCES PROVIDED CORRECTING WEAKNESSES
821                1  POAMS INCLUDE SECURITY WEAKNESSES DISCOVERED A...
822                1  COSTS ASSOCIATED REMEDIATING WEAKNESSES IDENTI...
823                1  PROGRAM OFFICIALS CONTRACTORS REPORT PROGRESS ...
824                1  ORGANIZATION ESTABLISHED REMOTE ACCESS PROGRAM...
825                1  DOCUMENTED POLICIES PROCEDURES AUTHORIZING MON...
826                1  PROTECTS UNAUTHORIZED CONNECTIONS SUBVERSION A...
827                1  USERS UNIQUELY IDENTIFIED AUTHENTICATED ACCESS...
828                1  TELECOMMUTING POLICY FULLY DEVELOPED NIST SECTION
829                1  APPLICABLE MULTIFACTOR AUTHENTICATION REQUIRED...
830                1  AUTHENTICATION MECHANISMS MEET NIST SPECIAL PU...
831                1  DEFINES IMPLEMENTS ENCRYPTION REQUIREMENTS INF...
832                1  REMOTE ACCESS SESSIONS ACCORDANCE OMB TIME

In [151]:
df.drop(['NQuestionText', 'QuestionText'], inplace=True, axis=1)

In [152]:
y = df['FK_QuestionType'].values 
X = df.drop(['FK_QuestionType' ], axis=1).values

In [153]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [154]:
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.974910394265233

In [155]:
def predict(s):
    arr=np.zeros(len(vectorizer.get_feature_names()), dtype=int)
    lst=s.split(' ')
    for w in lst:
        if w in vectorizer.get_feature_names():
            pos=vectorizer.get_feature_names().index(w)
            arr[pos]=1 
    return arr


In [156]:
arr=predict('PROVIDE NUMBER DNS NAMES SIGNED USING DNSSEC') 
prediction =model.predict([arr])
prediction[0]

2